In [1]:
import numpy as np
from time import time
from tqdm import tqdm
import pickle

In [2]:
with open("../../Downloads/result_levon.obj", "r") as f:
    content_pifu = f.readlines()

FileNotFoundError: [Errno 2] No such file or directory: '../../Downloads/result_levon.obj'

In [3]:
def get_vertices(mesh_path, with_texture = False):
    with open(mesh_path, "r") as f:
        content = f.readlines()
    vertices = [[np.float32(v_i) for v_i in v.split('\n')[0].split(' ')[1:]] 
                for v in content if v.startswith('v ')]
    vertices = np.array(vertices)
    if with_texture:
        return vertices
    else:
        return vertices[:,:3]

In [8]:
def mesh_verts_align(verts_1, verts_2, eps = 1e-8):
    "this function aligns verts_2 to verts_1"
    
    # finding bounding boxes
    bbox_1_x_min, bbox_1_x_max = np.min(verts_1[:,0]), np.max(verts_1[:,0])
    bbox_1_y_min, bbox_1_y_max = np.min(verts_1[:,1]), np.max(verts_1[:,1])
    bbox_1_z_min, bbox_1_z_max = np.min(verts_1[:,2]), np.max(verts_1[:,2])
    H1 = bbox_1_z_max - bbox_1_z_min
    W1 = bbox_1_y_max - bbox_1_y_min
    D1 = bbox_1_x_max - bbox_1_x_min
    
    bbox_2_x_min, bbox_2_x_max = np.min(verts_2[:,0]), np.max(verts_2[:,0])
    bbox_2_y_min, bbox_2_y_max = np.min(verts_2[:,1]), np.max(verts_2[:,1])
    bbox_2_z_min, bbox_2_z_max = np.min(verts_2[:,2]), np.max(verts_2[:,2])
    H2 = bbox_2_z_max - bbox_2_z_min
    W2 = bbox_2_y_max - bbox_2_y_min
    D2 = bbox_2_x_max - bbox_2_x_min
    
    # get_centers
    center_1 = 0.5* np.array([(bbox_1_x_min + bbox_1_x_max), 
                              (bbox_1_y_min + bbox_1_y_max),
                              (bbox_1_z_min + bbox_1_z_max)])

    center_2 = 0.5* np.array([(bbox_2_x_min + bbox_2_x_max), 
                              (bbox_2_y_min + bbox_2_y_max),
                              (bbox_2_z_min + bbox_2_z_max)])
    
#     center_1 = np.mean(verts_1, axis=0)
#     center_2 = np.mean(verts_2, axis=0)
    
    verts_2 = verts_2 - center_2
    verts_2[:,0] = verts_2[:,0]*(D1/D2+eps) 
    verts_2[:,1] = verts_2[:,1]*(W1/W2+eps)
    verts_2[:,2] = verts_2[:,2]*(H1/H2+eps)
    
    verts_2 = verts_2 + center_1
    
    return verts_2
    
#     bbox_2_x_min, bbox_2_x_max = np.min(verts_2[:,0]), np.max(verts_2[:,0])
#     bbox_2_y_min, bbox_2_y_max = np.min(verts_2[:,1]), np.max(verts_2[:,1])
#     bbox_2_z_min, bbox_2_z_max = np.min(verts_2[:,2]), np.max(verts_2[:,2])
#     H2 = bbox_2_z_max - bbox_2_z_min
#     W2 = bbox_2_y_max - bbox_2_y_min
#     D2 = bbox_2_x_max - bbox_2_x_min
    
#     center_1 = 0.5* np.array([(bbox_1_x_min + bbox_1_x_max), 
#                               (bbox_1_y_min + bbox_1_y_max),
#                               (bbox_1_z_min + bbox_1_z_max)])

#     center_2 = 0.5* np.array([(bbox_2_x_min + bbox_2_x_max), 
#                               (bbox_2_y_min + bbox_2_y_max),
#                               (bbox_2_z_min + bbox_2_z_max)])
    
#     return center_1, center_2, (H1, W1, D1), (H2, W2, D2)

In [9]:
verts_smpl = get_vertices('../smpl_tf.obj')

In [10]:
verts_pifu = get_vertices('../../Downloads/result_levon.obj')

In [46]:
verts_pifu_textures = get_vertices('../../Downloads/result_levon.obj', with_texture=True)[:,3:]

In [71]:
verts_smpl_random = get_vertices('../../Downloads/smpl_tf.obj')

In [11]:
verts_smpl_aligned = mesh_verts_align(verts_pifu, verts_smpl)

In [72]:
verts_smpl_random_aligned = mesh_verts_align(verts_pifu, verts_smpl_random)

In [12]:
verts_smpl_aligned.shape

(6890, 3)

In [93]:
def change_verts_in_mesh(mesh_path, new_verts, textures=False):
    with open(mesh_path, "r") as f:
        content = f.readlines()
        lines = content#.split('\n')
        new_content = []
        for i in tqdm(range(len(lines))):
            line = lines[i]
            if not line.startswith('v '):
                new_line = line
            else: 
                verts_part = ' '.join(line.split(' ')[:4])
                if textures:
                    new_line = line.replace(verts_part, 'v '+' '.join([str(val) for val in new_verts[i]]))
                else:
                    new_line = line.replace(verts_part, 'v '+' '.join([str(val) for val in new_verts[i]])+'\n')
            new_content.append(new_line)
    return ''.join(new_content)

In [67]:
new_content = change_verts_in_mesh('../smpl_tf.obj', verts_smpl_aligned)

100%|██████████| 20666/20666 [00:00<00:00, 437790.66it/s]


In [15]:
with open('../smpl_tf_aligned.obj', 'w') as file:
    file.write(new_content)

In [25]:
def pairwise_dist(A, B):
    # squared norms of each row in A and B
    na = np.sum(np.square(A), axis = 1)
    nb = np.sum(np.square(B), axis =  1)

    # na as a row and nb as a co"lumn vectors
    na = np.reshape(na, [-1, 1])
    nb = np.reshape(nb, [1, -1])

    # return pairwise euclidead difference matrix
    D = np.maximum(na - 2*np.matmul(A, B.T) + nb, 0.0)
    
    return D

In [43]:
start = time()
dists = pairwise_dist(verts_smpl_aligned, verts_pifu)
print(time()-start)

2.7966830730438232


In [44]:
dists.shape

(6890, 42500)

In [56]:
argmins = np.argmin(dists, axis=-1)

In [59]:
argmins

array([22225, 21578, 22682, ..., 14050, 14050, 14050])

In [60]:
smpl_textures = verts_pifu_textures[argmins]

In [62]:
def add_texture_to_mesh(non_textured_mesh_path, textures):
    with open(non_textured_mesh_path, "r") as f:
        content = f.readlines()
        lines = content#.split('\n')
        new_content = []
        for i in tqdm(range(len(lines))):
            line = lines[i]
            if not line.startswith('v '):
                new_line = line
            else: 
                verts_part = ' '.join(line.split(' ')[:4])
                rep = ' '+' '.join([str(val) for val in textures[i]])+'\n'
                new_line = line.replace('\n', rep)
            new_content.append(new_line)
    return ''.join(new_content)

In [69]:
new_content = add_texture_to_mesh('../smpl_tf_aligned.obj', textures=smpl_textures)

100%|██████████| 20666/20666 [00:00<00:00, 571956.84it/s]


In [70]:
with open('../smpl_tf_aligned_textured.obj', 'w') as file:
    file.write(new_content)

In [101]:
shifts = verts_smpl_random_aligned - verts_smpl_aligned

In [102]:
start = time()
dists = pairwise_dist(verts_pifu, verts_smpl_aligned)
print(time()-start)

4.7361297607421875


In [103]:
argmins = np.argmin(dists, axis=-1)

In [117]:
argmins.shape

(42500,)

In [105]:
verts_pifu_new = verts_pifu + shifts[argmins]

In [106]:
new_content = change_verts_in_mesh('../../Downloads/result_levon.obj', verts_pifu_new, textures=True)

100%|██████████| 127500/127500 [00:00<00:00, 556797.90it/s]


In [107]:
with open('../pifu_random.obj', 'w') as file:
    file.write(new_content)

In [99]:
content_pifu

['v -0.4700 0.7969 0.1562 0.6455 0.4662 0.3830\n',
 'v -0.4688 0.7969 0.1488 0.6391 0.4601 0.3765\n',
 'v -0.4688 0.7945 0.1562 0.6393 0.4623 0.3794\n',
 'v -0.4688 0.7920 0.1641 0.6463 0.4687 0.3844\n',
 'v -0.4714 0.7969 0.1641 0.6529 0.4736 0.3909\n',
 'v -0.4688 0.7907 0.1719 0.6482 0.4701 0.3850\n',
 'v -0.4726 0.7969 0.1719 0.6555 0.4759 0.3937\n',
 'v -0.4688 0.7904 0.1797 0.6496 0.4712 0.3859\n',
 'v -0.4735 0.7969 0.1797 0.6546 0.4744 0.3928\n',
 'v -0.4688 0.7900 0.1875 0.6502 0.4714 0.3857\n',
 'v -0.4743 0.7969 0.1875 0.6536 0.4728 0.3916\n',
 'v -0.4688 0.7908 0.1953 0.6567 0.4781 0.3924\n',
 'v -0.4743 0.7969 0.1953 0.6526 0.4714 0.3902\n',
 'v -0.4688 0.7922 0.2031 0.6678 0.4899 0.4044\n',
 'v -0.4741 0.7969 0.2031 0.6514 0.4697 0.3887\n',
 'v -0.4688 0.7934 0.2109 0.6761 0.4989 0.4136\n',
 'v -0.4725 0.7969 0.2109 0.6596 0.4785 0.3961\n',
 'v -0.4688 0.7943 0.2188 0.6810 0.5045 0.4195\n',
 'v -0.4713 0.7969 0.2188 0.6639 0.4829 0.3996\n',
 'v -0.4688 0.7954 0.2266 0.686

In [110]:
with open('/Users/test/Desktop/hmr/models/neutral_smpl_with_cocoplus_reg.pkl', 'rb') as f:
    dd = pickle.load(f, encoding="latin-1")

In [116]:
dd['weights'].shape

(6890, 24)

In [118]:
weights_for_pifu = dd['weights'][argmins]

In [119]:
weights_for_pifu.shape

(42500, 24)

In [153]:
np.save('weights_for_pifu.npy', dd['weights'])

In [173]:
np.save('verts_pifu.npy', verts_smpl)

In [266]:
basketbolist_smpl = np.load('basketbolist_smpl.npy')[0]

In [267]:
# basketbolist_smpl[:,1] = -basketbolist_smpl[:,1]
# basketbolist_smpl[:,2] = -basketbolist_smpl[:,2]

In [268]:
new_content = change_verts_in_mesh('../smpl_tf.obj', basketbolist_smpl)

100%|██████████| 20666/20666 [00:00<00:00, 421461.64it/s]


In [269]:
with open('../basketbolist_smpl.obj', 'w') as file:
    file.write(new_content)

In [270]:
verts_pifu_reversed = verts_pifu.copy()
verts_pifu_reversed[:,1] = -verts_pifu_reversed[:,1]
verts_pifu_reversed[:,2] = -verts_pifu_reversed[:,2]

new_content = change_verts_in_mesh('../../Downloads/result_levon.obj', verts_pifu_reversed, textures=True)
with open('../delete_me.obj', 'w') as file:
    file.write(new_content)

100%|██████████| 127500/127500 [00:00<00:00, 576897.05it/s]


In [271]:
verts_pifu_reversed_aligned = mesh_verts_align(basketbolist_smpl, verts_pifu_reversed)

In [273]:
new_content = change_verts_in_mesh('../../Downloads/result_levon.obj', verts_pifu_reversed_aligned, textures=True)
with open('../delete_me.obj', 'w') as file:
    file.write(new_content)

100%|██████████| 127500/127500 [00:00<00:00, 517716.05it/s]


In [274]:
np.save('verts_pifu_reversed_aligned.npy', verts_pifu_reversed_aligned)

In [278]:
start = time()
dists = pairwise_dist(verts_pifu_reversed_aligned, basketbolist_smpl)
print(time()-start)
argmins = np.argmin(dists, axis=-1)

blend_weights_for_pifu = dd['weights'][argmins]

3.0656089782714844


In [279]:
np.save('weights_for_pifu.npy', blend_weights_for_pifu)

In [294]:
pifu_zero = np.load('/Users/test/Desktop/hmr/pifu_another_pose.npy')[0]
new_content = change_verts_in_mesh('../../Downloads/result_levon.obj', pifu_zero, textures=True)
with open('../pifu_another_pose.obj', 'w') as file:
    file.write(new_content)

100%|██████████| 127500/127500 [00:00<00:00, 568310.92it/s]
